## Compounds produced by commmunity

In [4]:
import pandas as pd
import reframed
import os
import json
import sys

sys.path.append('../functions')

import translation_dicts
import general_functions as general_func

In [5]:
compounds_dict, source_dict,substrate_dict, gas_sheet_dict, community_dict = translation_dicts.translation_dicts()

### Make overview of all compounds measured the enrichment cultures

The strategy for defining if the compound is produced or not is:
- Assuming that positive concertation==the compound is produced by the community

In [6]:
community_production = {}

# For each source 
for name, id_source in source_dict.items():
    
    # Read and process excel sheet
    ## Get the Excel sheet specifically for the source
    data_df = pd.read_excel("../input/files_from_fairdomhub/enrichment_cultures _data.xlsx",sheet_name=name)
    data_df.dropna(how='all', axis='columns',inplace=True)
    
    data_df = data_df.iloc[:,1:-1]
    data_df.set_index("Sample name ",inplace=True)
    data_df.columns = [col.replace(" ","") for col in data_df.columns]
    
    # For each substrate
    for name_sub,id_sub in substrate_dict.items():
        community_id = id_source+"_"+id_sub
                
        if community_id=="M_A": # Marshland on Avicel is not a combination in our experiments.
            continue
        
        community_production[community_id]={}
        data_df_sub_max = data_df[data_df.index.str.contains(community_id)].max()
        
        # For each compound in our analysis
        for compound in compounds_dict.keys():
            if compound not in data_df_sub_max.index:
                print(community_id)
                print(compound + " not in columns")
                continue
            
            # If max concentration is higher than 0 -> assume that it is produced by a community member
            if data_df_sub_max[compound]>0:
                community_production[community_id]["R_EX_"+compounds_dict[compound]+"_e"]=1
                
        # These gasses where observed for all enrichment cultures. (for gasses CH4 is not included because it is not a part of the bacterial universal model)
        community_production[community_id]["R_EX_h2_e"]=1
        community_production[community_id]["R_EX_co2_e"]=1

### Save data 

In [7]:
with open("../output/community_production.json", "w") as outfile: 
    json.dump(community_production, outfile)